## **Sub-problem 1 with Knn and C++**

**Etape 1:** On installe ANN qui sera indispensable pour compiler nos codes C++

In [ ]:
import os
os.chdir('/content/data-anonymization/sub-problem1/k-nearest-neighbour/code-C++')
!unzip ann_1.1.2.zip
os.chdir('/content/data-anonymization/sub-problem1/k-nearest-neighbour/code-C++/ann_1.1.2')
!make linux-g++
!cp -r /content/data-anonymization/sub-problem1/k-nearest-neighbour/code-C++/ann_1.1.2/include /usr
!cp -r /content/data-anonymization/sub-problem1/k-nearest-neighbour/code-C++/ann_1.1.2/bin /usr
!cp -r /content/data-anonymization/sub-problem1/k-nearest-neighbour/code-C++/ann_1.1.2/lib /usr

**Etape 2:** On convertit les labelles des données d'entraînement en 0 ou 1. le label "I-PER" devient 1 et les autres deviennent 0.

In [5]:
import pandas as  pd
def transform(x):
    if "PER" in x:
        return 1
    return 0
df = pd .read_csv("/content/data-anonymization/data/CONLL2003/eng.testb.true_labels.csv",header=None)
df =df[0].apply(transform)
testb_label = pd.DataFrame(df)
df = pd .read_csv("/content/data-anonymization/data/CONLL2003/eng.testa.true_labels.csv",header=None)
df =df[0].apply(transform)
testa_label = pd.DataFrame (df)
df = pd .read_csv("/content/data-anonymization/data/CONLL2003/eng.train.true_labels.csv",header=None)
df =df[0].apply(transform)
train_label = pd.DataFrame (df)

**Etape 3:** On fusionne les labelles et les répresentations dans un meme fichier pour pouvoir le passer en argument de notre programme C++

In [6]:
train_repr=pd.read_csv('/content/data-anonymization/data/CONLL2003/eng.train.representation.csv',header=None)
test_a_rep=pd.read_csv('/content/data-anonymization/data/CONLL2003/eng.testa.representation.csv',header=None)
test_b_rep=pd.read_csv('/content/data-anonymization/data/CONLL2003/eng.testb.representation.csv',header=None)

In [7]:
train = pd.concat([train_label,train_repr],axis=1)
testa = pd.concat([testa_label,test_a_rep],axis=1)
testb = pd.concat([testb_label,test_b_rep],axis =1)

In [8]:
train.to_csv('/content/data-anonymization/data/CONLL2003/eng.train.csv',index=False,header=None)
testa.to_csv('/content/data-anonymization/data/CONLL2003/eng.testa.csv',index=False,header=None)
testb.to_csv('/content/data-anonymization/data/CONLL2003/eng.testb.csv',index=False,header=None)

**Etape 4:** On compile et on teste notre classifieur KNN pour $k=3$ 

In [48]:
os.chdir('/content/data-anonymization/sub-problem1/k-nearest-neighbour/code-C++')
!make knn_classifier

g++ -std=c++11 -O2 -I/usr/local/eigen-3.3.7 -I/usr/local/ann-1.1.2/include -Wall -Wextra knn_classifier.cpp ConfusionMatrix.o KnnClassification.o Classification.o Dataset.o -o knn_classifier -L/usr/local/ann-1.1.2/lib -lANN 


**testa**

In [14]:
!./knn_classifier 3 /content/data-anonymization/dataCONLL2003/eng.train.csv /content/data-anonymization/data/CONLL2003/eng.testa.csv

No column specified for classification, assuming first column of dataset (0).
Dataset with 5000 samples, and 769 dimensions.
Computing k-NN classification (k=3, classification over column 0)...
Prediction and Confusion Matrix filling

execution time: 26660ms

		Predicted
		0	1
Actual	0	4654	21
	1	67	258

Error rate		0.0176
False alarm rate	0.00449198
Detection rate		0.793846
F-score			0.854305
Precision		0.924731


**testb**

In [15]:
!./knn_classifier 3 /content/data-anonymization/data/CONLL2003/eng.train.csv /content/data-anonymization/data/CONLL2003/eng.testb.csv

No column specified for classification, assuming first column of dataset (0).
Dataset with 5000 samples, and 769 dimensions.
Computing k-NN classification (k=3, classification over column 0)...
Prediction and Confusion Matrix filling

execution time: 25530ms

		Predicted
		0	1
Actual	0	4667	27
	1	65	241

Error rate		0.0184
False alarm rate	0.00575202
Detection rate		0.787582
F-score			0.839721
Precision		0.899254


**Etape 5:** On teste d'autres valeur de $k$ sur testb pour obtenir celui avec le meilleur F-score

**$k=4$**

In [16]:
!./knn_classifier 4 /content/data-anonymization/data/CONLL2003/eng.train.csv /content/data-anonymization/data/CONLL2003/eng.testb.csv

No column specified for classification, assuming first column of dataset (0).
Dataset with 5000 samples, and 769 dimensions.
Computing k-NN classification (k=4, classification over column 0)...
Prediction and Confusion Matrix filling

execution time: 27483ms

		Predicted
		0	1
Actual	0	4675	19
	1	116	190

Error rate		0.027
False alarm rate	0.00404772
Detection rate		0.620915
F-score			0.737864
Precision		0.909091


On constate que le F-score est plus mauvais. Donc on ne va pas aller chercher les valeurs de $k$ plus grandes. Par contre on va regarder pour $k=2$

$k=2$

In [17]:
!./knn_classifier 2 /content/data-anonymization/data/CONLL2003/eng.train.csv /content/data-anonymization/data/CONLL2003/eng.testb.csv

No column specified for classification, assuming first column of dataset (0).
Dataset with 5000 samples, and 769 dimensions.
Computing k-NN classification (k=2, classification over column 0)...
Prediction and Confusion Matrix filling

execution time: 25238ms

		Predicted
		0	1
Actual	0	4679	15
	1	108	198

Error rate		0.0246
False alarm rate	0.00319557
Detection rate		0.647059
F-score			0.763006
Precision		0.929577


On constate que le F-score pour $k=2$ est plus mauvais que  pour $k=3$. D'où le $k$ optimal est $k=3$

**Etape 7:** On enreigistre la prediction dans un fichier de sortie

In [ ]:
!./knn_classifier 3 /content/data-anonymization/data/CONLL2003/eng.train.csv /content/data-anonymization/data/CONLL2003/eng.testb.csv 0 /content/data-anonymization/data/CONLL2003/eng.testb-predicted.csv